In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [4]:
#reading data from CSV file
dataset = pd.read_csv('health_data.csv')
dataset  = dataset.sample(frac = 1)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1:].values
datasize = X.shape[0]
X_train = X[:(datasize*7)//10,:]
y_train = y[:(datasize*7)//10,:]
X_test = X[(datasize*7)//10:,:]
y_test =  y[(datasize*7)//10:,:]
print(dataset)

     age  restbps  chol  category
471   55      156   246         1
306   47      128   265         0
192   24       95   208         0
114   21       96   227         0
46    28      104   253         0
..   ...      ...   ...       ...
528   74      149   331         1
81    29      108   159         0
278   31      113   167         0
576   61      139   392         1
568   75      131   339         1

[700 rows x 4 columns]


In [5]:
X_train_0 = X_train[y_train[:,0]==0]
X_train_1 = X_train[y_train[:,0]==1]
print(X_train_1)

[[ 55 156 246]
 [ 58 156 332]
 [ 57 128 287]
 [ 53 142 296]
 [ 66 131 345]
 [ 68 156 378]
 [ 38 136 232]
 [ 61 127 351]
 [ 53 143 189]
 [ 65 143 346]
 [ 69 163 364]
 [ 66 145 348]
 [ 72 137 310]
 [ 41 137 262]
 [ 65 154 383]
 [ 49 142 302]
 [ 55 133 291]
 [ 67 158 395]
 [ 48 139 273]
 [ 46 151 322]
 [ 63 145 367]
 [ 42 135 212]
 [ 42 132 280]
 [ 69 137 311]
 [ 67 135 302]
 [ 53 135 274]
 [ 56 149 348]
 [ 53 142 256]
 [ 38 138 249]
 [ 61 137 322]
 [ 65 143 384]
 [ 57 139 247]
 [ 61 143 433]
 [ 47 139 259]
 [ 71 135 344]
 [ 44 132 184]
 [ 54 123 143]
 [ 51 142 294]
 [ 66 147 351]
 [ 42 140 201]
 [ 64 145 355]
 [ 54 142 245]
 [ 53 143 292]
 [ 59 161 296]
 [ 65 159 351]
 [ 48 131 226]
 [ 42 134 153]
 [ 51 109 242]
 [ 62 146 309]
 [ 39 131 239]
 [ 65 135 354]
 [ 34 135 251]
 [ 45 146 251]
 [ 58 153 328]
 [ 64 126 359]
 [ 67 153 364]
 [ 61 135 417]
 [ 48 141 261]
 [ 56 151 272]
 [ 55 161 322]
 [ 47 111 202]
 [ 46 126 226]
 [ 34 135 238]
 [ 70 150 333]
 [ 56 157 287]
 [ 55 161 309]
 [ 44 146 

In [6]:
#Multivariate gaussian distribution
def Multivariate_gaussian(inp,mean,covariance):
    det_covariance = np.linalg.det(covariance)
    front = ((2*np.pi)**3)*det_covariance
    front = pow(front, 0.5)
    front = 1/front
    exp = np.matmul(np.transpose(inp - mean),np.linalg.inv(covariance))
    exp = np.matmul(exp,inp-mean)
    exp = np.exp(-0.5*exp)
    ret = front*exp
    return ret

In [7]:
#Calculating the mean and Variance for the class 0
N_0 = X_train_0.shape[0]
mean_0 = (1/N_0)*np.sum(X_train_0, axis = 0)
print(mean_0)
covariance_0 = (1/N_0)*(np.matmul(np.transpose(X_train_0-mean_0),X_train_0 - mean_0))
print(covariance_0)

[ 32.25539568 113.82733813 228.02517986]
[[ 81.31247089  91.75273019 -25.31938047]
 [ 91.75273019 164.0709073  -35.37694736]
 [-25.31938047 -35.37694736 940.58569691]]


In [8]:
#Calculating the mean and covariance matrix for the class 1
N_1 = X_train_1.shape[0]
mean_1 = (1/N_1)*np.sum(X_train_1, axis = 0)
covariance_1 = (1/N_1)*(np.matmul(np.transpose(X_train_1-mean_1),X_train_1 - mean_1))
print(mean_1)
print(covariance_1)

[ 54.66509434 141.58018868 291.49528302]
[[  89.42085707   38.55279904  433.20360894]
 [  38.55279904  140.44168298  287.42490655]
 [ 433.20360894  287.42490655 3656.05186454]]


In [9]:
def predict(inp):
    p_0 = Multivariate_gaussian(inp,mean_0,covariance_0)
    p_1 = Multivariate_gaussian(inp,mean_1,covariance_1)
    if p_0 > p_1:
        return 0
    else:
        return 1

In [13]:
def Accuracy_of_classifier(test_X,test_y):
    count = 0
    score = [[0,0] for i in range(2)]
    for i in range(test_X.shape[0]):
        pred_cat = predict(test_X[i])
        score[test_y[i][0]][pred_cat]+=1
        if test_y[i][0]==pred_cat:
            count+=1
    print("tp: {}".format(score[0][0]),"tn: {}".format(score[1][1]),"fp: {}".format(score[0][1]), "fn: {}".format(score[1][0]))
    precision = score[1][1]/(score[0][1]+score[1][1])
    recall = score[1][1]/(score[1][0]+score[1][1])
    F1 = (2*precision*recall)/(precision+recall)
    print("Accuracy: {} ".format(count/len(test_X)))
    print("Precision: {}".format(precision))
    print("Recall: {}".format(recall))
    print("F1 Score: {}".format(F1))


In [14]:
Accuracy_of_classifier(X_train,y_train)

tp: 239 tn: 181 fp: 39 fn: 31
Accuracy: 0.8571428571428571 
Precision: 0.8227272727272728
Recall: 0.8537735849056604
F1 Score: 0.837962962962963


In [15]:
Accuracy_of_classifier(X_test, y_test)

tp: 104 tn: 76 fp: 18 fn: 12
Accuracy: 0.8571428571428571 
Precision: 0.8085106382978723
Recall: 0.8636363636363636
F1 Score: 0.8351648351648351
